---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    top_125 = df.str.extract(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})')[:125]
    top125_193 = df.str.extract(r'(\d{1,2})[/-]?[.,]?[\s]?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[/-]?[.,]?[\s]?(\d{2,4})')[125:194]
    top194_227=df.str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[/-]?[.,]?[\s]?(\d{1,2})[/-]?[.,]?[\s]?([12]\d{3})')[194:228]
    top228_342 = df.str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[/-]?[.,]?[\s]?(\d\d)?([12]\d{3})')[228:343]
    top343_500 = df.str.extract('(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)?(\d{1,2})?[/-]?([12]\d{3})')[343:]

    def sort_columns(variable):
        variable = variable[[1,0,2]]
        variable = variable.rename(columns={1:0, 0:1, 2:2})
        return variable

    top125_193 = sort_columns(top125_193)
    top343_500 = sort_columns(top343_500)


    top228_342 = top228_342.fillna(1)
    top343_500 = top343_500.fillna(1)



    month_dict = {'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,'September': 9, 'Mar': 3,
                  'November': 11, 'Jul': 7, 'January': 1,'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6,
                  'March': 3, 'July': 7,  'Nov': 11, 'April': 4, 'Decemeber': 12, 'December':12}

    def replace_month(variable):
        variable.iloc[:,0] = variable.iloc[:,0].replace(month_dict)
        return variable
    
    top125_193 = replace_month(top125_193)
    top194_227 = replace_month(top194_227)
    top228_342 = replace_month(top228_342)


    
    datedf = pd.concat([top_125,top125_193, top194_227, top228_342, top343_500])

    datedf = datedf.apply(lambda x: str(x[0])+'-'+str(x[1])+'-'+str(x[2]),axis=1)

    datedf = pd.to_datetime(datedf)

    return pd.Series(datedf.sort_values().index)
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [3]:
# reg1 = (r'(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})') 
# reg2 = (r'((?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]?[,]? (?:\d{1,2}[,]?[\s]?)?\d{2,4})')
# reg3 = (r'\d{1,2}[/][1|2]\d{3}')
# reg4 = (r'[1|2]\d{3}')
# # finalized